In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [4]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'State code',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [5]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [6]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,Current Status,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,KL,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


# Data cleaning and transformations

In [7]:
# shape of dataframe
# ==================

p_df.shape

(12060, 26)

In [8]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [9]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Backup Notes']

In [10]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
State code                                   0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Type of transmission                         0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [11]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(12060, 19)
Patient Number 	 106
State Patient Number 	 10017
Date Announced 	 548
Age Bracket 	 10576
Gender 	 9698
Detected City 	 10611
Detected District 	 2439
Detected State 	 549
State code 	 549
Current Status 	 549
Notes 	 2275
Contracted from which Patient (Suspected) 	 10656
Nationality 	 10781
Type of transmission 	 9142
Status Change Date 	 659
Source_1 	 739
Source_2 	 9143
Source_3 	 11736
Backup Notes 	 11700


In [12]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(12060, 19)


Patient Number                                 106
State Patient Number                         10017
Date Announced                                 548
Age Bracket                                  10576
Gender                                        9698
Detected City                                10611
Detected District                             2439
Detected State                                 549
State code                                     549
Current Status                                 549
Notes                                         2275
Contracted from which Patient (Suspected)    10656
Nationality                                  10781
Type of transmission                          9142
Status Change Date                             659
Source_1                                       739
Source_2                                      9143
Source_3                                     11736
Backup Notes                                 11700
dtype: int64

In [13]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

(11511, 19)


Patient Number                                   0
State Patient Number                          9468
Date Announced                                   0
Age Bracket                                  10028
Gender                                        9150
Detected City                                10062
Detected District                             1891
Detected State                                   0
State code                                       0
Current Status                                   1
Notes                                         1727
Contracted from which Patient (Suspected)    10107
Nationality                                  10233
Type of transmission                          8594
Status Change Date                             111
Source_1                                       191
Source_2                                      8595
Source_3                                     11188
Backup Notes                                 11151
dtype: int64

In [14]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')

In [15]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
4129,4130,NaN,05/04/2020,NaN,NaN,NaN,Lucknow,Uttar Pradesh,UP,Hospitalized,Details Awaited,NaN,NaN,NaN,05/04/2020,UP State bulletin,NaN,NaN,NaN
6123,6124,NaN,09/04/2020,NaN,NaN,NaN,Mumbai,Maharashtra,MH,Hospitalized,Details awaited,NaN,NaN,NaN,09/04/2020,https://twitter.com/PTI_News/status/1248126814...,NaN,NaN,NaN
3006,3007,NaN,03/04/2020,NaN,NaN,NaN,Mumbai,Maharashtra,MH,Hospitalized,NaN,NaN,NaN,NaN,03/04/2020,https://twitter.com/PTI_News/status/1246097072...,https://experience.arcgis.com/experience/8167a...,NaN,NaN
6778,6779,NaN,10/04/2020,NaN,NaN,NaN,Ahmadabad,Gujarat,GJ,Hospitalized,Details awaited,NaN,NaN,NaN,10/04/2020,https://twitter.com/ANI/status/124848110017683...,https://gujcovid19.gujarat.gov.in/,NaN,NaN
6244,6245,NaN,09/04/2020,NaN,NaN,NaN,Shupiyan,Jammu and Kashmir,JK,Hospitalized,Details awaited,NaN,NaN,NaN,09/04/2020,https://twitter.com/kansalrohit69/status/12482...,NaN,NaN,NaN


In [16]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

Index(['patient_number', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'state_code', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'type_of_transmission',
       'status_change_date', 'source_1', 'source_2', 'source_3',
       'backup_notes', 'p_id'],
      dtype='object')

In [17]:
p_df = p_df.loc[:, :'backup_notes']

# Saving Data

In [18]:
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
2815,2816,TN-P382,03/04/2020,NaN,NaN,NaN,Viluppuram,Tamil Nadu,TN,Hospitalized,Travelled to Delhi,E0,NaN,Local,03/04/2020,https://twitter.com/ANI/status/124604385644717...,https://twitter.com/ANI/status/124605490629894...,NaN,NaN
1555,1556,TN-P75,31/03/2020,NaN,M,NaN,Kanniyakumari,Tamil Nadu,TN,Hospitalized,Travelled to Delhi,E0,NaN,Local,31/03/2020,https://twitter.com/Vijayabaskarofl/status/124...,NaN,NaN,NaN
3896,3897,TN-P546,05/04/2020,NaN,M,NaN,Salem,Tamil Nadu,TN,Hospitalized,Travelled to Delhi,E0,NaN,Local,05/04/2020,State Bulletin: 05 Apr Evening,NaN,NaN,NaN
7882,7883,NaN,11/04/2020,NaN,NaN,NaN,Srinagar,Jammu and Kashmir,JK,Hospitalized,Details awaited,NaN,NaN,NaN,11/04/2020,https://twitter.com/kansalrohit69/status/12489...,NaN,NaN,NaN
7370,7371,NaN,10/04/2020,NaN,NaN,NaN,Mumbai,Maharashtra,MH,Hospitalized,Details awaited,NaN,NaN,NaN,10/04/2020,https://twitter.com/rajeshtope11/status/124864...,NaN,NaN,NaN


In [19]:
p_df.to_csv('patients_data.csv', index=False)